In [0]:
from __future__ import print_function

import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
num_classes = 10
epochs = 50

img_rows, img_cols = 28, 28

################## Def dataset ###########################

#the data, split between train and test sets

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#x_train = x_train.reshape(60000, 56)
#x_test = x_test.reshape(10000, 56)

if K.image_data_format() == 'channels first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)
  
  
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#to ensure a good dynamic, help the learnig convergence
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#convert class vectors to binary class matrices

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

################## Data augmentation ###################

################## Def model ###########################

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0,25))
model.add(Conv2D(32, (5,5), activation='relu')) # meilleur sans les deux couches suivantes
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu')) ##
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2))) ##

#Epoch 00050: val_acc did not improve from 0.91760
#Test loss:  0.6874618720195256
#Test accuracy:  0.9066

#Avec batchNormalization
#Epoch 00050: val_acc did not improve from 0.91740
#Test loss:  0.6105943850596436
#Test accuracy:  0.9144

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0,5))
model.add(Dense(num_classes, activation='softmax'))

#model.compile(loss=keras.losses.categorical_crossentropy,
#            optimizer=keras.optimizers.Adadelta(),
#             metrics=['accuracy'])
#Epoch 00050: val_acc did not improve from 0.91970
#Test loss:  0.6960430979938712
#Test accuracy:  0.917
  
  
model.summary()

model.compile(loss='categorical_crossentropy', optimizer = RMSprop(), metrics=['accuracy'])
##Epoch 00050: val_acc did not improve from 0.92170
##Test loss:  0.7203312129669707
##Test accuracy:  0.9142

################## Learning ###########################

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)


# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#fit permet de lancer l'apprentissage, avec les donnees, les labels, la taille du batch, le nb d'epoque, la verbosite, et les donnees de validation d'epoque a epoque
history = model.fit(x_train, y_train, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     verbose=1, 
                     validation_data=(x_test, y_test),
                     callbacks=callbacks_list)

#evaluate toward test dataset
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

model.save('Test_FASHION_MNIST_MANO.h5')

60000 train samples
10000 test samples
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_14 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
batch_normalization_15 (Batc (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_28 (Conv

In [0]:
# https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow(
    x_train),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow(
    x-test),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

ajouter ça pour la data augmentation!

In [0]:
#model from : https://www.dlology.com/blog/one-simple-trick-to-train-keras-model-faster-with-batch-normalization/
#Epoch 00050: val_acc did not improve from 0.90950
#Test loss:  0.7327184855340101
#Test accuracy:  0.902

#En rajoutant Dense 128
#Epoch 00050: val_acc did not improve from 0.91210
#Test loss:  0.7428650218252558
#Test accuracy:  0.9072

#En rajoutant Dense 256
#Total params: 246,008
#Epoch 00050: val_acc did not improve from 0.91430
#Test loss:  0.6275242215685546
#Test accuracy:  0.9134

#Dernier résultat :
#Epoch 00050: val_acc did not improve from 0.91700
#Test loss:  0.7110278423965914
#Test accuracy:  0.9069
  
from __future__ import print_function

import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
num_classes = 10
epochs = 50

img_rows, img_cols = 28, 28

################## Def dataset ###########################

#the data, split between train and test sets

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#x_train = x_train.reshape(60000, 56)
#x_test = x_test.reshape(10000, 56)

if K.image_data_format() == 'channels first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)
  
  
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#to ensure a good dynamic, help the learnig convergence
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#convert class vectors to binary class matrices

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

################## Data augmentation ###################

################## Def model ###########################

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu')) # meilleur sans les deux couches suivantes
model.add(BatchNormalization())
model.add(Flatten())


#Epoch 00050: val_acc did not improve from 0.91760
#Test loss:  0.6874618720195256
#Test accuracy:  0.9066

#Avec batchNormalization
#Epoch 00050: val_acc did not improve from 0.91740
#Test loss:  0.6105943850596436
#Test accuracy:  0.9144

model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

#model.compile(loss=keras.losses.categorical_crossentropy,
#            optimizer=keras.optimizers.Adadelta(),
#             metrics=['accuracy'])
#Epoch 00050: val_acc did not improve from 0.91970
#Test loss:  0.6960430979938712
#Test accuracy:  0.917
  
  
model.summary()

model.compile(loss='categorical_crossentropy', optimizer = RMSprop(), metrics=['accuracy'])
##Epoch 00050: val_acc did not improve from 0.92170
##Test loss:  0.7203312129669707
##Test accuracy:  0.9142

################## Learning ###########################

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)


# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#fit permet de lancer l'apprentissage, avec les donnees, les labels, la taille du batch, le nb d'epoque, la verbosite, et les donnees de validation d'epoque a epoque
history = model.fit(x_train, y_train, 
                     batch_size=batch_size, 
                     epochs=epochs, 
                     verbose=1, 
                     validation_data=(x_test, y_test),
                     callbacks=callbacks_list)

#evaluate toward test dataset
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

model.save('Test_FASHION_MNIST_MANO.h5')

60000 train samples
10000 test samples
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_27 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
batch_normalization_28 (Batc (None, 11, 11, 64)        256       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_41 (Con

Comparer les deux solutions ci dessus et en choisir une.
Ajouter de la dataAugmentation dedans.